## FINAL REPORT (FR)

## Due 29 April 2016 (Friday, 11:59pm)

**Important Note:** Before uploading your midterm project on Canvas, please name your file as following:

*MT#_FirstLastName.ipynb*

where "#" denotes the midterm number, "FirstLastName" is the name of the student. Students are allowed to work in groups (2 or max. of 3 students). **Each student will hand in their own file**. If you work with another student, please write her/his name on top of the first cell (in a Markdown cell).

**Question (30 points): CHOOSE YOUR OWN COMPUTATIONAL INTELLIGENCE APPLICATION**

In this last exercise, you will choose your own CI application from one of the following main applications (Surrogate-based optimization can be coupled with the other two):

* Game AI
* 3D Printing
* Surrogate-based Optimization

You are already familiar with Game AI and 3D printing applications. You can get some ideas about the surrogate-based optimization from the following three papers (you can download them from [UT library](http://www.lib.utexas.edu/) with your EID):

* Y. Jin, [A comprehensive survey of fitness approximation in evolutionary computation](http://link.springer.com/article/10.1007%2Fs00500-003-0328-5), Soft Computing, Vol:9, Issue:1, 2005.
* A.I.J. Forrester, A.J. Keane, [Recent advances in surrogate-based optimization](http://www.sciencedirect.com/science/article/pii/S0376042108000766), Progress in Aerospace Sciences, Vol:45, 50-79, 2009.
* Y. Jin, [Surrogate-assisted evolutionary computation: Recent advances and future challenges](http://www.sciencedirect.com/science/article/pii/S2210650211000198), Swarm and Evolutionary Computation, 61-70, 2011.

One of the recent papers that we worked on can be found in this [link](https://docs.google.com/viewer?a=v&pid=sites&srcid=ZGVmYXVsdGRvbWFpbnxjZW1jdHV0dW18Z3g6MmVmY2Q1YjA0ZWVjNzE3MQ).

Some other interesting projects could be, but **not limited to**:

* Evolutionary multi-objective optimization (EMO) and its applications in games or 3D printing
* Evolutionary Many-objective optimization
* Use of different evolutionary algorithms: Genetic Programming, Evolution Strategies, Particle Swarm Optimization, Differential Evolution, CMA-ES, Estimation of Distribution Algorithms, etc. (most of these algorithms are avilable in DEAP)
* Approximation of 3D objects with cubes, spheres or any base hypercubes using evolutionary algorithms (needs integration of DEAP with OpenSCAD or programming an EA in OpenSCAD)
* Designing a 2D car with EAs to finish a given track in shorted amount of time (requires a physics engine)
* 3D printable walking or jumping objects (requires a physics engine)
* Designing 3D printable accessories using EAs (aesthetic measure is needed for the fitness calculation)
* Surrogate-based optimization using a physical simulation or analytical engineering design problem.
* Surrogate-based EMO.
* Surrogate-based optimization in high-dimensional search space (more than 15 or 20 dimensions).
* Robust optimization -- Optimization under uncertainties. For instance, you can investigate the variablity in 3D printing of gears and how to incorporate these variances while designing a reliable gear mechanism
* 3D printable lamp design --incorporating variable wall thickness (to control translucency). It may require a physics engine.

**IMPORTANT NOTES:** 

* You can discuss your final project with your friends or mentors, but you have to discuss about it with the instructor before working on it.
* Prepare your report in the following format.
* Write your report below this cell, not as part of the explanations of format or content.

**////////////////////////////////////////////////////////////////**

**FORMAT OF THE REPORT:**

# On Minimizing Support Structures Using Surrogate Based Methods

## Abstract 


In situations where calls to the cost function are costly, using traditional techniques such as genetic algorithms can be unfeasible. In this report, we expolore the field of surrogate-based optimization using the example minimizing the amount of support structures required to 3D-print a model. We use an Radial Base Function (RBF) in order to model the cost function.

## Introduction

In this project, we expand upon our previous project **[3]** by finding an optimum cutting plane for a 3D object in order to minimize the amount of support structures while not restricting ourselves to planes parallel to the y-z axis. This drastically increases the search space for the genetic algorithm and thus makes it infeasible, especially since calculating the amount of support structures can take time on the order of 15-20 seconds for a single individual. Naively using this cost function for a genetic algorithm can mean that it could take days to run (depending on parameters such as population size). Thus, in this project, we aim to remove this bottleneck by approximating the function using a Radial Basis Function, which is much faster to compute.

Radial Basis Functions (RBFs) are those functions which's value depends only upon the the norm of the input. ie. $$\phi(x) \sim \Phi(|x|)$$
This norm is usually defined as the Euclidean distance of the vector from the center and this is what we use, though other definitions are possible. Some examples of radial basis functions are the Gaussian function and the multiquadric function. In the following examples, we let $r = || x - c ||$, where $x$ is an arbitrary point and $c$ is the center point.

Multiquadric function:
$$\phi(r) = \sqrt{1+ (\epsilon r)^2}$$

Radial Basis Functions have been used for "expensive black-box global optimization" before **[6]**. In **[6]**, the RBF interpolant is defined as the following. Given $f: V \to F$, where $V$ is the space of possible inputs and $F$ is a space of scalars, and distinct points $(x_1, f(x_1)), (x_2, f(x_2)), \ldots, (x_n, f(x_n)) \in V \times F$, the interpolant has the following form:

$$ s_n(x) = \sum\limits_{i=1}^{n} \lambda_i \phi(||x_i - c||) + p(x)$$

In this formula, $p(x)$ is a polynomial specific to the choice of $\phi$. For the multiquadratic function, $p(x) = 0$. Finding the values of $\lambda_i$ "trains" the RBF network to approximate values of $f$.

We shall now discuss how we used the RBF interpolation technique in order to minimize the amount of support structures in printing our 3D model.


* First paragraph: A short description (3-5 sentences) about your project.
* Second paragraph: A *detailed* description of the problem, related work found in the literature. A summary of the structure of your report (what will you be discussing in the upcoming sections?)
* Please don't forget to provide any references (**with corresponding numbers**) supporting your sentences.

## Methodology

### Encoding the data

The code base we were given used six design variables in order to specify which cutting plane to use. We managed to chop the number of design varibles to just 3 in the following manner:

* The unit vector

 This is composed of 3 values; $x$, $y$ and $z$ , which specify the direction of the unit vector perpendicular to the plane. In reality, we only need the x and the y values since we know that $$z = \left|\sqrt{x^2 + y^2}\right|$$ . **[a]**
 

 
* The distance
 The distance of the plane from the origin in the direction of the unit vector.
 
This is called the Hessian Normal form and we can translate back and forth between this representation and the one used in the original implementation.

### Getting the data

We use existing code in slicing.cpp to get the data to train our RBF neural network. We vary x and y between -1 with a step size of 0.2, and z between 0 and 100 with a step size of 10. This gives us ~250 training examples using which we can then train our function

For later comparisons, we recorded the parameters which gave us the lowest cost. We found the global minimum to be at the point $(a,b,c) = (0.6, 0.8, 0), d = 70$, with a cost of 16. This is what the cut looks like:

<img src="http://i.imgur.com/IoAbBIg.jpg"></img>

### Implementing the RBF Network

While we coded our own implementation of an RBF as an exercise (using SciPy's implementation as a reference), we mostly used SciPy's implementation as it allows for many more possibilities. For example, it allows us to choose from many different kinds of Radial Based Functions, which is good for experimentation.

SciPy's implementation also allows one to specify many parameters, two of which are very interesting to us. These are:

* $\epsilon$ : Specifies how much influence each training example has on the final function
* Smooth : Quantizes the regularization in order to prevent overfitting.

### The 1 dimensional case

For the problem that we are presenting, it makes sense to break it down into parts. Hence, we begin by considering the one dimensional case, in which we only vary the x value while keeping the y and l constant. After training, we got the following graph when we set $\epsilon = 0.1$ and $smooth = 1$. 

<img src="http://i.imgur.com/t996iD9.png">

In this graph, the green dots represent data that the network was trained on and red dots represent unseen data. Now, when we change the smooth function to 0, we get the following graph, which is clearly overfitting the data.

<img src="http://i.imgur.com/BhMRznw.png">

Clearly, it would be quite easy to use the first model to get the cost for each individual in a genetic algorithm quickly, whereas the same would have taken 10-15 seconds *(per individual!)* using slicing.cpp

### The 2 dimensional case

We did an exercise similar to the 1D case but now in two dimensions, keeping the l values constant but varying the x and the y value. This is the graph that we obtained in this case. The light green nodes represnt the data that we used to train the function while the dark green nodes represent the unseen data.

* You are expected to use an evolutionary algorithm; please provide the details of your implementation (which operators you use?, how do you calculate the fitness values?, etc.)
* Please provide the statistics about your calculations.
* If you use a simulation, physics engine, analytical test problem or parametric design for 3D printing, give the detailed description of that.
* You can provide example figures about the problem.

**Results and Discussions**:

* You can provide a figure showing the fitness values versus generations. 
* Results obtained with different algorithm settings.

### Conclusions

The data that we used for this experiment signifies a fairly complex situation. The potential of RBFs in modelling complex and computationally expensive cost functions for prediction and optimization purposes is quite clear from how they were able to efficiently fit this data. Our model was bale to produce the minima for the values as expected and as can seen from the graphs, were able to predict the values for unseen data as expected. 

While we were not able to finish the three dimensional version of this problem in the given time frame, we are quite confident that we would have been able to accomplish it, simply by looking at the results obtained form the 1D and 2D cases. Perhaps we should have tried to develop some ways to visualize the data in 3 dimensions before proceeding with our approach. It might have also been useful to play around with other paramaters of the RBF. Again, we do not believe that this says anything about the viability of RBF networks for modelling tasks.



* Wrap up your work like you do in Abstract section and provide detailed summary of the results. 
* If your approach didn't work, you can still give the arguments to help potential readers to avoid the same mistakes.

**References**:

**[1]** A. I.J. Forrester, A. J. Keane, *"Recent advances in surrogate-based optimization"*, University of Southampton, Pg 10, 2009

* List your citations with the order of: Author1, Author2, "Title of the article", Name of the Journal or Conference that the paper published at, Pages, Year. 
* Start your references with numbers and use those numbers in the text.

## Appendix A


## Footnotes

**[a]** We do not consider negative square roots because of an initial bug in our implementation while getting data

**THANK YOU**